In [1]:
import sys

from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torchvision.transforms import Resize
import torch

sys.path.insert(0, '..')

## View MobileNetV3 Architecture for Reference

In [2]:
weights = MobileNet_V3_Large_Weights.IMAGENET1K_V2
model = mobilenet_v3_large(weights=weights)

print(model)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

## Import `SiameseDuplicateImageNetwork` for Testing

In [3]:
from scripts.dataset_duplicate_image import DuplicateImageDataset
from scripts.model import SiameseDuplicateImageNetwork

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
model = SiameseDuplicateImageNetwork().to(device)

In [6]:
print(model)

SiameseDuplicateImageNetwork(
  (preprocess): ImageClassification(
      crop_size=[224]
      resize_size=[232]
      mean=[0.485, 0.456, 0.406]
      std=[0.229, 0.224, 0.225]
      interpolation=InterpolationMode.BILINEAR
  )
  (model): Sequential(
    (0): SiameseNetwork(
      (net1): MobileNetV3(
        (features): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): Hardswish()
          )
          (1): InvertedResidual(
            (block): Sequential(
              (0): Conv2dNormActivation(
                (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
                (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
              )
              (1): 

In [7]:
train_dataset = DuplicateImageDataset('../data/Training Data', transforms=[Resize((224, 224), antialias=True)])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [8]:
feature1, feature2, _ = next(iter(train_dataloader))
feature1, feature2

(tensor([[[[223, 226, 226,  ..., 235, 233, 232],
           [224, 225, 225,  ..., 233, 233, 233],
           [226, 224, 224,  ..., 233, 233, 233],
           ...,
           [109, 112, 113,  ..., 191, 195, 196],
           [109, 111, 112,  ..., 180, 189, 196],
           [108, 110, 109,  ..., 175, 183, 188]],
 
          [[231, 229, 229,  ..., 239, 237, 236],
           [230, 229, 229,  ..., 237, 237, 237],
           [229, 229, 230,  ..., 237, 237, 237],
           ...,
           [ 88,  87,  86,  ..., 110, 117, 119],
           [ 86,  84,  83,  ...,  99, 109, 118],
           [ 82,  81,  80,  ...,  99, 101, 109]],
 
          [[228, 228, 228,  ..., 238, 236, 235],
           [228, 228, 228,  ..., 236, 236, 236],
           [228, 228, 228,  ..., 236, 236, 236],
           ...,
           [ 61,  62,  61,  ...,  23,  19,  17],
           [ 60,  60,  59,  ...,  23,  20,  17],
           [ 57,  57,  56,  ...,  22,  24,  14]]],
 
 
         [[[228, 228, 230,  ..., 153, 157, 159],
         

In [9]:
feature1 = feature1.to(device)
feature2 = feature2.to(device)

output = model(feature1, feature2)
print(output)

C:\Users\ruiqinng\miniconda3\envs\dai\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor([[-0.1215],
        [-0.1164],
        [-0.1181],
        [-0.1170],
        [-0.1200],
        [-0.1226],
        [-0.1176],
        [-0.1212],
        [-0.1121],
        [-0.1140],
        [-0.1175],
        [-0.1151],
        [-0.1193],
        [-0.1201],
        [-0.1135],
        [-0.1176],
        [-0.1211],
        [-0.1211],
        [-0.1211],
        [-0.1174],
        [-0.1246],
        [-0.1204],
        [-0.1166],
        [-0.1110],
        [-0.1205],
        [-0.1172],
        [-0.1171],
        [-0.1219],
        [-0.1170],
        [-0.1205],
        [-0.1158],
        [-0.1176],
        [-0.1235],
        [-0.1160],
        [-0.1168],
        [-0.1197],
        [-0.1255],
        [-0.1215],
        [-0.1211],
        [-0.1174],
        [-0.1189],
        [-0.1150],
        [-0.1214],
        [-0.1247],
        [-0.1218],
        [-0.1167],
        [-0.1147],
        [-0.1215],
        [-0.1211],
        [-0.1194],
        [-0.1210],
        [-0.1204],
        [-0.

In [10]:
output.shape

torch.Size([64, 1])